In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Disable tensorflow debugging logs
import tensorflow as tf
import matplotlib.pyplot as plt

## Importar dataset

In [2]:
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [3]:
dataset = tfds.load('imdb_reviews', as_supervised=True)

In [4]:
raw_train_ds, raw_test_ds = dataset['train'], dataset['test']

In [5]:
for text, label in raw_train_ds.take(1):
    print(text.numpy(), label.numpy())

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it." 0


## Pipeline

In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BUFFER_SIZE = tf.data.experimental.cardinality(raw_train_ds)
BUFFER_SIZE.numpy()

25000

In [7]:
def clean_text(raw_text, label):
    lowercase = tf.strings.lower(raw_text)
    clean = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return clean, label

batch_size = 16
voc_size = 5000
seq_length = 20

train_ds = raw_train_ds.map(clean_text).shuffle(BUFFER_SIZE).batch(
        batch_size, num_parallel_calls=AUTOTUNE).prefetch(
        AUTOTUNE)


test_ds = raw_test_ds.map(clean_text).batch(
        batch_size, num_parallel_calls=AUTOTUNE).prefetch(
        AUTOTUNE)

In [8]:
for i in train_ds.take(1):
    print(i)

(<tf.Tensor: shape=(16,), dtype=string, numpy=
array([b"i remember i saw this cartoon when i was 6 or 7. my grandfather picked up the video of it for free at the mall. i remember that it really sucked. the plot had no sense. i hated the fox that became casper's friend. he was so stupid! casper cried his head off if he couldn't find a friend. so what? get over it! the only good part and i don't want to sound mean-spirited was when the fox got shot and died at the end. i laughed my head off in payback because this cartoon sucked so much. the bad news is the fox resurrects and becomes a ghost. i wish he had stayed dead. i think i even gave the video of this to somebody because i hated it. no wonder they were offering it for free at the mall. if you have a child don't let them watch this. they will probably agree with me that it sucks.",
       b"i have never commented on imdb before, but i feel i have to after watching the batman animation. its absolute rubbish! warner brothers had the pe

## Definir modelo

In [9]:
elmo = hub.load("https://tfhub.dev/google/elmo/2").signatures["default"]
inputs = tf.constant(["This is my dog", "My friend"])
elmo(inputs)["elmo"]

<tf.Tensor: shape=(2, 4, 1024), dtype=float32, numpy=
array([[[-0.514227  , -0.4635899 ,  0.06965357, ..., -0.74261266,
         -0.03954674,  0.04713609],
        [-0.27030078, -0.51212245,  0.11180548, ..., -0.18113986,
          0.8950255 ,  0.6264232 ],
        [-0.16576877, -0.2522542 ,  0.10615169, ...,  0.2372579 ,
          0.09109694, -0.00911216],
        [ 0.05849284, -0.67593753,  0.57834536, ...,  0.06305956,
         -0.18194857,  0.4078228 ]],

       [[-0.0501115 , -0.2674372 , -0.49513027, ...,  0.03255737,
         -0.0942485 , -0.32873583],
        [-0.8822049 , -1.1591299 , -0.3480692 , ...,  0.15842313,
          0.05209573,  0.253683  ],
        [-0.02840755, -0.04354125,  0.04128971, ...,  0.02580839,
         -0.01432125, -0.01650512],
        [-0.02840755, -0.04354125,  0.04128971, ...,  0.02580839,
         -0.01432125, -0.01650512]]], dtype=float32)>

In [10]:
elmo_rnn = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/elmo/2", trainable=False),
    tf.keras.layers.Embedding(
        input_dim=voc_size, output_dim=128),
    tf.keras.layers.SimpleRNN(128),
    tf.keras.layers.Dense(1)
])

- Probar rnn con batch de prueba

In [11]:
elmo_rnn(inputs)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.02728894],
       [0.03234689]], dtype=float32)>

- Información del modelo

In [12]:
elmo_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1024)              93600852  
                                                                 
 embedding (Embedding)       (None, 1024, 128)         640000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 94,273,877
Trainable params: 673,025
Non-trainable params: 93,600,852
_________________________________________________________________


## Entrenamiento 

In [13]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [14]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001)

In [15]:
metrics = tf.metrics.BinaryAccuracy()

In [16]:
len(test_ds), tf.data.experimental.cardinality(test_ds)

(1563, <tf.Tensor: shape=(), dtype=int64, numpy=1563>)

In [17]:
elmo_rnn.compile(optimizer=opt,
                 loss=loss,
                 metrics=metrics)

In [18]:
epochs = 3

In [ ]:
history = elmo_rnn.fit(train_ds,
                       validation_data=test_ds,
                       epochs=epochs)

Epoch 1/3
 309/1563 [====>.........................] - ETA: 44:40 - loss: 0.6941 - binary_accuracy: 0.4869

- ELMo contiene aprendizaje que se puede aprovechar para obtener mejor accuracy, sin embargo, consiste en un gran número de parámetros, lo que implica que el entrenamiento requiera de mayor memoria y tiempo de procesamiento.

## Ejercicio

- Utilizar TFEecords https://www.tensorflow.org/tutorials/load_data/tfrecord para guardar los embeddings y acelerar el entrenamiento.